In [1]:
from langchain_openai import ChatOpenAI
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field


llm = ChatOpenAI(temperature=0, model_name="gpt-4.1-mini")

In [2]:
email_conversation = """From: 김철수 (chulsoo.kim@bikecorporation.me)
To: 이은채 (eunchae@teddyinternational.me)
Subject: "ZENESIS" 자전거 유통 협력 및 미팅 일정 제안

안녕하세요, 이은채 대리님,

저는 바이크코퍼레이션의 김철수 상무입니다. 최근 보도자료를 통해 귀사의 신규 자전거 "ZENESIS"에 대해 알게 되었습니다. 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 혁신과 품질을 선도하는 기업으로, 이 분야에서의 장기적인 경험과 전문성을 가지고 있습니다.

ZENESIS 모델에 대한 상세한 브로슈어를 요청드립니다. 특히 기술 사양, 배터리 성능, 그리고 디자인 측면에 대한 정보가 필요합니다. 이를 통해 저희가 제안할 유통 전략과 마케팅 계획을 보다 구체화할 수 있을 것입니다.

또한, 협력 가능성을 더 깊이 논의하기 위해 다음 주 화요일(1월 15일) 오전 10시에 미팅을 제안합니다. 귀사 사무실에서 만나 이야기를 나눌 수 있을까요?

감사합니다.

김철수
상무이사
바이크코퍼레이션
"""

In [3]:
from itertools import chain
from langchain_core.prompts import PromptTemplate

prompt = PromptTemplate.from_template(
    "다음의 이메일 내용중 중요한 내용을 추출해 주세요.\n\n{email_conversation}"
)

In [4]:

chain = prompt | llm

In [15]:
result = chain.stream({"email_conversation" :email_conversation})

In [16]:
from langchain_core.messages import AIMessageChunk
def stream_response(response, return_output=False):
    """
    AI 모델로부터의 응답을 스트리밍하여 각 청크를 처리하면서 출력합니다.

    이 함수는 `response` 이터러블의 각 항목을 반복 처리합니다. 항목이 `AIMessageChunk`의 인스턴스인 경우,
    청크의 내용을 추출하여 출력합니다. 항목이 문자열인 경우, 문자열을 직접 출력합니다. 선택적으로, 함수는
    모든 응답 청크의 연결된 문자열을 반환할 수 있습니다.

    매개변수:
    - response (iterable): `AIMessageChunk` 객체 또는 문자열일 수 있는 응답 청크의 이터러블입니다.
    - return_output (bool, optional): True인 경우, 함수는 연결된 응답 문자열을 문자열로 반환합니다. 기본값은 False입니다.

    반환값:
    - str: `return_output`이 True인 경우, 연결된 응답 문자열입니다. 그렇지 않으면, 아무것도 반환되지 않습니다.
    """
    answer = ""
    for token in response:
        if isinstance(token, AIMessageChunk):
            answer += token.content
            print(token.content, end="", flush=True)
        elif isinstance(token, str):
            answer += token
            print(token, end="", flush=True)
    if return_output:
        return answer

In [17]:
output = stream_response(result, return_output=True)

다음은 이메일의 중요한 내용입니다:

- 발신자: 김철수 상무(바이크코퍼레이션)
- 수신자: 이은채 대리(테디인터내셔널)
- 목적: "ZENESIS" 자전거 유통 협력 논의 및 미팅 일정 제안
- 요청 사항: ZENESIS 자전거의 상세 브로슈어(기술 사양, 배터리 성능, 디자인 정보)
- 미팅 제안: 1월 15일 화요일 오전 10시, 귀사 사무실에서 미팅 희망
- 배경: 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 경험과 전문성을 보유하고 있음

In [18]:
output

'다음은 이메일의 중요한 내용입니다:\n\n- 발신자: 김철수 상무(바이크코퍼레이션)\n- 수신자: 이은채 대리(테디인터내셔널)\n- 목적: "ZENESIS" 자전거 유통 협력 논의 및 미팅 일정 제안\n- 요청 사항: ZENESIS 자전거의 상세 브로슈어(기술 사양, 배터리 성능, 디자인 정보)\n- 미팅 제안: 1월 15일 화요일 오전 10시, 귀사 사무실에서 미팅 희망\n- 배경: 바이크코퍼레이션은 자전거 제조 및 유통 분야에서 경험과 전문성을 보유하고 있음'

In [20]:
class EmailSummary(BaseModel):
    person: str = Field(description="메일을 보낸 사람")
    email: str = Field(description="메일을 보낸 사람의 이메일 주소")
    subject: str = Field(description="메일 제목")
    summary: str = Field(description="메일 본문을 요약한 텍스트")
    date: str = Field(description="메일 본문에 언급된 미팅 날짜와 시간")


# PydanticOutputParser 생성
parser = PydanticOutputParser(pydantic_object=EmailSummary)

In [23]:
print(parser.get_format_instructions())

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required": ["person", "email", "subject", "summary", "date"]}
```


In [24]:
prompt = PromptTemplate.from_template(
    """
You are a helpful assistant. Please answer the following questions in KOREAN.

QUESTION:
{question}

EMAIL CONVERSATION:
{email_conversation}

FORMAT:
{format}
"""
)

# format 에 PydanticOutputParser의 부분 포맷팅(partial) 추가
prompt = prompt.partial(format=parser.get_format_instructions())

In [25]:
prompt

PromptTemplate(input_variables=['email_conversation', 'question'], input_types={}, partial_variables={'format': 'The output should be formatted as a JSON instance that conforms to the JSON schema below.\n\nAs an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}\nthe object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.\n\nHere is the output schema:\n```\n{"properties": {"person": {"description": "메일을 보낸 사람", "title": "Person", "type": "string"}, "email": {"description": "메일을 보낸 사람의 이메일 주소", "title": "Email", "type": "string"}, "subject": {"description": "메일 제목", "title": "Subject", "type": "string"}, "summary": {"description": "메일 본문을 요약한 텍스트", "title": "Summary", "type": "string"}, "date": {"description": "메일 본문에 언급된 미팅 날짜와 시간", "title": "Date", "type": "string"}}, "required

In [26]:
chain = prompt | llm 

In [27]:
response = chain.stream({
    'question' : "이메일 내용중 주요 내용을 추출해 주세요",
    'email_conversation' : email_conversation
})

In [28]:
output = stream_response(response, return_output=True)

```json
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "바이크코퍼레이션 김철수 상무가 ZENESIS 자전거에 대한 상세 브로슈어(기술 사양, 배터리 성능, 디자인)를 요청하고, 유통 전략과 마케팅 계획 수립을 위해 협력 가능성을 논의하고자 1월 15일 화요일 오전 10시에 미팅을 제안함.",
  "date": "1월 15일 화요일 오전 10시"
}
```

In [30]:
print(output)

```json
{
  "person": "김철수",
  "email": "chulsoo.kim@bikecorporation.me",
  "subject": "\"ZENESIS\" 자전거 유통 협력 및 미팅 일정 제안",
  "summary": "바이크코퍼레이션 김철수 상무가 ZENESIS 자전거에 대한 상세 브로슈어(기술 사양, 배터리 성능, 디자인)를 요청하고, 유통 전략과 마케팅 계획 수립을 위해 협력 가능성을 논의하고자 1월 15일 화요일 오전 10시에 미팅을 제안함.",
  "date": "1월 15일 화요일 오전 10시"
}
```


In [34]:
print(parser.parse(output))

person='김철수' email='chulsoo.kim@bikecorporation.me' subject='"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안' summary='바이크코퍼레이션 김철수 상무가 ZENESIS 자전거에 대한 상세 브로슈어(기술 사양, 배터리 성능, 디자인)를 요청하고, 유통 전략과 마케팅 계획 수립을 위해 협력 가능성을 논의하고자 1월 15일 화요일 오전 10시에 미팅을 제안함.' date='1월 15일 화요일 오전 10시'


In [43]:
parser.parse(output).person

'김철수'

In [51]:
output[8:-4]

'{\n  "person": "김철수",\n  "email": "chulsoo.kim@bikecorporation.me",\n  "subject": "\\"ZENESIS\\" 자전거 유통 협력 및 미팅 일정 제안",\n  "summary": "바이크코퍼레이션 김철수 상무가 ZENESIS 자전거에 대한 상세 브로슈어(기술 사양, 배터리 성능, 디자인)를 요청하고, 유통 전략과 마케팅 계획 수립을 위해 협력 가능성을 논의하고자 1월 15일 화요일 오전 10시에 미팅을 제안함.",\n  "date": "1월 15일 화요일 오전 10시"\n}'

In [52]:
import json 

json.loads(output[8:-4])


{'person': '김철수',
 'email': 'chulsoo.kim@bikecorporation.me',
 'subject': '"ZENESIS" 자전거 유통 협력 및 미팅 일정 제안',
 'summary': '바이크코퍼레이션 김철수 상무가 ZENESIS 자전거에 대한 상세 브로슈어(기술 사양, 배터리 성능, 디자인)를 요청하고, 유통 전략과 마케팅 계획 수립을 위해 협력 가능성을 논의하고자 1월 15일 화요일 오전 10시에 미팅을 제안함.',
 'date': '1월 15일 화요일 오전 10시'}